In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OrdinalEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.datasets import fetch_openml

import skrub
from skrub import tabular_learner, DropCols, MinHashEncoder, TableVectorizer, AggJoiner, TableReport
from skrub import _selectors as s

import skore
from skore import EstimatorReport
from skore_hub_project.project.project import Project

Fetch the dataset. We will use the credit fraud dataset from skrub.

In [2]:
X, y = fetch_openml("adult", version=2, as_frame=True, return_X_y=True)

In [3]:
skrub.TableReport(X)

Processing column  14 / 14


,,,,,,,,,,,,,,


In [4]:
y.value_counts()

class
<=50K    37155
>50K     11687
Name: count, dtype: int64

In [5]:
y = 1*(y == ">50K")

In [6]:
X_train, X_test, y_train, y_test = skore.train_test_split(X, y, random_state=1)

╭───────────────────────────── HighClassImbalanceWarning ──────────────────────────────╮
│ It seems that you have a classification problem with a high class imbalance. In this │
│ case, using train_test_split may not be a good idea because of high variability in   │
│ the scores obtained on the test set. To tackle this challenge we suggest to use      │
│ skore's CrossValidationReport with the `cv_splitter` parameter of your choice.       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────── ShuffleTrueWarning ─────────────────────────────────╮
│ We detected that the `shuffle` parameter is set to `True` either explicitly or from  │
│ its default value. In case of time-ordered events (even if they are independent),    │
│ this will result in inflated model performance evaluation because natural drift will │
│ not be taken into account. We recommend setting the shuffle parameter to `False` in  │
│ order to ensure the evaluation process is really representative of your production   │
│ release process.                                                                     │
╰──────────────────────────────────────────────────────────────────────────────────────╯

Simpler is better.
Let's do a simple baseline.

In [7]:
baseline = tabular_learner('classification')
baseline

Pipeline(steps=[('tablevectorizer',
                 TableVectorizer(high_cardinality=MinHashEncoder(),
                                 low_cardinality=ToCategorical())),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier())])

In [8]:
baseline_report = EstimatorReport(baseline, X_train = X_train, y_train=y_train, X_test = X_test, y_test = y_test)
baseline_report.help()

╭───────────── Tools to diagnose estimator HistGradientBoostingClassifier ─────────────╮
│ EstimatorReport                                                                      │
│ ├── .metrics                                                                         │
│ │   ├── .accuracy(...)         (↗︎)     - Compute the accuracy score.                 │
│ │   ├── .brier_score(...)      (↘︎)     - Compute the Brier score.                    │
│ │   ├── .log_loss(...)         (↘︎)     - Compute the log loss.                       │
│ │   ├── .precision(...)        (↗︎)     - Compute the precision score.                │
│ │   ├── .precision_recall(...)         - Plot the precision-recall curve.            │
│ │   ├── .recall(...)           (↗︎)     - Compute the recall score.                   │
│ │   ├── .roc(...)                      - Plot the ROC curve.                         │
│ │   ├── .roc_auc(...)          (↗︎)     - Compute the ROC AUC score.                  │
│ │   ├── .timings(...)                  - Get all measured processing times related   │
│ │   │   to the estimator.                                                            │
│ │   ├── .custom_metric(...)            - Compute a custom metric.                    │
│ │   └── .report_metrics(...)           - Report a set of metrics for our estimator.  │
│ ├── .feature_importance                                                              │
│ │   └── .permutation(...)              - Report the permutation feature importance.  │
│ ├── .cache_predictions(...)            - Cache estimator's predictions.              │
│ ├── .clear_cache(...)                  - Clear the cache.                            │
│ ├── .get_predictions(...)              - Get estimator's predictions.                │
│ └── Attributes                                                                       │
│     ├── .X_test                        - Testing data                                │
│     ├── .X_train                       - Training data                               │
│     ├── .y_test                        - Testing target                              │
│     ├── .y_train                       - Training target                             │
│     ├── .estimator_                    - The cloned or copied estimator              │
│     ├── .estimator_name_               - The name of the estimator                   │
│     ├── .fit_time_                     - The time taken to fit the estimator, in     │
│     │   seconds                                                                      │
│     └── .ml_task                       - No description available                    │
│                                                                                      │
│                                                                                      │
│ Legend:                                                                              │
│ (↗︎) higher is better (↘︎) lower is better                                             │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [9]:
baseline_report.metrics.report_metrics()

HistGradientBoostingClassifier
Metric           Label / Average                                
Precision        0                                      0.898685
                 1                                      0.789835
Recall           0                                      0.943779
                 1                                      0.665082
ROC AUC                                                 0.930531
Brier score                                             0.086700
Fit time (s)                                            0.819292
Predict time (s)                                        0.087698

In [10]:
# create project
project = Project(name="project demo - census", tenant="Probabl")

In [ ]:
project.put("baseline", baseline_report)

Let's go a bit further in that baseline by optimizing the parameters.

In [ ]:
from sklearn.model_selection import GridSearchCV

baseline_2 = GridSearchCV(
                estimator = baseline,
                param_grid = {
                    "randomforestclassifier__max_leaf_nodes": np.arange(10, 2_000, 30),
                    "randomforestclassifier__n_estimators": [100, 200],
                    "randomforestclassifier__max_depth": [None, 10, 20],
                },
                cv = 5,
                n_jobs = -1,
                refit=True,
                scoring="neg_log_loss",
            )

In [ ]:
baseline_2 = tabular_learner("classification")
baseline_report_2 = EstimatorReport(baseline_2, X_train = X_train, y_train=y_train, X_test = X_test, y_test = y_test)

In [ ]:
project.put("baseline_2", baseline_report_2)

In [ ]:
baseline_report_2.metrics.report_metrics()

In [ ]:
comp = skore.ComparisonReport([baseline_report, baseline_report_2])
comp.help()

In [ ]:
comp.metrics.report_metrics(pos_label = 1)

DEMO PART 2 - after superior review